Imports

In [49]:
import os
import time
import requests
from datetime import datetime
from typing import Dict, Any
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def timed_operation(operation_name):
    def decorator(func):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            logging.info(f"Starting {operation_name}...")
            result = func(*args, **kwargs)
            elapsed_time = time.time() - start_time
            logging.info(f"Completed {operation_name} in {elapsed_time:.2f} seconds.")
            return result

        return wrapper

    return decorator

In [50]:
# 1. Загрузка аудио
@timed_operation("Загрузка аудио")
def load_audio(audio_path: str) -> str:
    """
    Загружает аудиофайл по указанному пути.

    Args:
        audio_path (str): Путь к аудиофайлу.

    Returns:
        str: Путь к файлу, если файл существует.

    Raises:
        FileNotFoundError: Если аудиофайл не найден.
    """
    if os.path.exists(audio_path):
        logger.info(f"Загружен аудиофайл: {audio_path}")
        return audio_path
    else:
        logger.error(f"Файл не найден: {audio_path}")
        raise FileNotFoundError(f"Файл не найден: {audio_path}")

# 2. Транскрипция и диаризация через WhisperX
@timed_operation("Транскрипция и диаризация через WhisperX")
def transcribe_and_diarize(audio_path: str, whisperx_endpoint: str, whisperx_params: Dict[str, Any]) -> Dict[str, Any]:
    """
    Выполняет транскрипцию и диаризацию аудиофайла с помощью сервиса WhisperX.

    Args:
        audio_path (str): Путь к аудиофайлу.
        whisperx_endpoint (str): URL-адрес для вызова API WhisperX.
        whisperx_params (Dict[str, Any]): Параметры запроса для WhisperX.

    Returns:
        Dict[str, Any]: Результаты транскрипции и диаризации в формате JSON.

    Raises:
        Exception: Если запрос к WhisperX завершился ошибкой.
    """
    logger.info("Отправка аудиофайла на транскрипцию и диаризацию в WhisperX...")
    with open(audio_path, 'rb') as audio_file:
        files = {'file': audio_file}
        response = requests.post(whisperx_endpoint, files=files, params=whisperx_params)

        if response.status_code == 200:
            transcription_result = response.json()
            logger.info("Транскрипция и диаризация выполнены успешно.")
            return transcription_result
        else:
            logger.error(f"Ошибка транскрипции и диаризации: {response.status_code}, {response.text}")
            raise Exception(f"Не удалось выполнить транскрипцию и диаризацию: {response.status_code}, {response.text}")

# 3. Инференс через Llama.cpp
@timed_operation("инференс через Llama.cpp")
def llama_cpp_inference(transcription_result: Dict[str, Any], llama_cpp_endpoint: str) -> Dict[str, Any]:
    """
    Отправляет результат транскрипции на модель Llama.cpp для инференса.

    Args:
        transcription_result (Dict[str, Any]): Результаты транскрипции и диаризации.
        llama_cpp_endpoint (str): URL-адрес для вызова API модели Llama.cpp.

    Returns:
        Dict[str, Any]: Результат работы модели Llama.cpp.

    Raises:
        Exception: Если запрос к Llama.cpp завершился ошибкой.
    """
    logger.info(f"Отправка данных на инференс модели Llama.cpp... {llama_cpp_endpoint}")
    system_promt= """
        <|user|>
Вы - ИИ секретарь, чья задача конспектировать происходящие на различных рабочих встречах
Задача:
Создание структурированного и краткого протокола на основе JSON с субтитрами разговора с рабочих встреч.

Инструкции:
1. Создание точных транскрипций, без добавления лишней информации, с указанием только ключевых тем обсуждений и задач.
2. Распределение задач с указанием только ответственных лиц и сроков выполнения.
3. Формирование структурированных и кратких протоколов, включающих:
   - Список участников и их роли (только тех, кто активно принимал участие).
   - Повестку дня с кратким описанием обсуждаемых вопросов.
   - Задачи с указанием ответственных и сроков.
   - Дополнительные заметки только по сути.
   - Исключайте любые дублирования информации, ошибки или излишние детали.

Формат ответа:
{
    "name_report": "Совещание в государственной думе",
    "document_type": "pdf",
    "password": "pdf",
    "data": {
      "date": "2024-09-07T21:10:52.564Z",
      "time": "21:10:52.564Z",
      "duration": "P3D",
      "participants": [
        "SPEAKER_00",
        "SPEAKER_01",
      ],
      "agenda": [
        "1. Обсуждение итогов прошедшего дня и ощущений от совещания.",
      ],
      "blocks": [
        {
          "name_block": "Задачи",
          "proposals": [
            {
              "text": "Сделать все возможное для защиты экономики",
              "context": "Не определен",
              "audio_time": {
                "start": 133.98,
                "end": 140.025
              }
            },
            {
              "text": "нужно собраться и нужно организовать работу",
              "context": "Сегодня",
              "audio_time": {
                "start": 171.105,
                "end": 182.45
              }
            }
          ]
        },
        {
          "name_block": "Обратная связь",
          "proposals": [
            {
              "text": "- SPEAKER_01 признал, что нужно нужно собраться и организовать работку как надо",
              "context": "",
              "audio_time": {
                "start": 171.105,
                "end": 182.45
              }
            }
          ]
        }
      ],
      "audio_times": [
        {
          "start": 0,
          "end": 182.45
        }
      ]
    }
  }

На вход подается только JSON с субтитрами в следующем формате:
{
    "result": {
        "segments": [
          {
            "start": 1.088,
            "end": 12.716,
            "text": " Вы знаете, что в Государственную Думу у меня вынесено предложение о назначении вас на должность председателя правительства Российской Федерации.",
            "speaker": "SPEAKER_01"
          },
          {
            "start": 12.756,
            "end": 20.001,
            "text": "Совсем недавно мы встречались с коллегами и оценивали работу правительства за предыдущие годы.",
            "speaker": "SPEAKER_01"
          },
          {
            "start": 20.061,
            "end": 28.807,
            "text": "Сделано в сложных условиях немало, и мне кажется, что было бы правильно, если бы",
            "speaker": "SPEAKER_01"
          },
          {
            "start": 36.504,
            "end": 42.048,
            "text": " Мы с вами говорили и о структуре, говорили о персонале.",
            "speaker": "SPEAKER_01"
          },
          {
            "start": 42.088,
            "end": 44.89,
            "text": "В целом, думаю, мы на правильном пути.",
            "speaker": "SPEAKER_01"
          },
          {
            "start": 44.95,
            "end": 65.183,
            "text": "И очень надеюсь на то, что депутаты Государственной Думы, а вы не так давно были в Госдуме, отчитывались, они знают, что и как правительство, и вами, как председателям правительства, сделано за последние годы, оценят должным образом и поддержат вас в ходе ваших консультаций предстоящих сегодня в Госдуме.",
            "speaker": "SPEAKER_01"
          },
          {
            "start": 94.77,
            "end": 97.152,
            "text": " Спасибо, уважаемый Владимир Владимирович.",
            "speaker": "SPEAKER_00"
          },
          {
            "start": 97.192,
            "end": 113.905,
            "text": "Хочу в первую очередь поблагодарить вас за доверие, которое вы оказали мне, за задачи, которые вы поставили перед Федеральным Собранием в своем послании, и, конечно, те национальные цели развития, которые были указаны в новом майском указе.",
            "speaker": "SPEAKER_00"
          },
          {
            "start": 114.885,
            "end": 118.468,
            "text": " Это ориентир и приоритеты в работе правительства.",
            "speaker": "SPEAKER_00"
          },
          {
            "start": 118.508,
            "end": 121.871,
            "text": "Хочу вас заверить, что никаких пауз в работе правительства не будет.",
            "speaker": "SPEAKER_00"
          },
          {
            "start": 121.911,
            "end": 124.593,
            "text": "Мы будем продолжать текущую работу.",
            "speaker": "SPEAKER_00"
          },
          {
            "start": 124.613,
            "end": 133.92,
            "text": "Также считаю, что мы должны обеспечить преемственность по всем национальным целям, которые были до этого, 204 и 474 указе.",
            "speaker": "SPEAKER_00"
          },
          {
            "start": 133.98,
            "end": 140.025,
            "text": "Сделаем все для развития нашей экономики, чтобы оправдать доверие наших людей.",
            "speaker": "SPEAKER_00"
          },
          {
            "start": 140.065,
            "end": 144.208,
            "text": "И уверен, что под вашим руководством мы все задачи, которые поставлены, решим.",
            "speaker": "SPEAKER_00"
          },
          {
            "start": 145.229,
            "end": 151.012,
            "text": " Мы с вами вместе и с коллегами из правительства формулировали национальные цели развития.",
            "speaker": "SPEAKER_01"
          },
          {
            "start": 151.093,
            "end": 158.578,
            "text": "Это, конечно, главное, к чему мы должны стремиться, к реализации этих целей по всем направлениям.",
            "speaker": "SPEAKER_01"
          },
          {
            "start": 158.658,
            "end": 163.461,
            "text": "И, как показывает практика последних лет, в целом у нас…",
            "speaker": "SPEAKER_01"
          },
          {
            "start": 166.082,
            "end": 171.025,
            "text": " Получается добиваться тех результатов, которые нужны стране.",
            "speaker": "SPEAKER_01"
          },
          {
            "start": 171.105,
            "end": 182.45,
            "text": "А в сегодняшних непростых условиях, конечно, нужно собраться и нужно организовать работу именно так, как мы с вами договорились на последней встрече с правительством, работать без пауз.",
            "speaker": "SPEAKER_01"
          }
        ]
      }
    }
}
На выход должен выдаваться только желаемый JSON.
<|end|>
<|assistant|>
        """
    # Set headers for the request
    headers = {
    "Content-Type": "application/json"
    }
    data = {
    "prompt": f"{system_promt} : {transcription_result}",
    "max_tokens": 2048  # Limit the number of tokens generated
            }
    response = requests.post(llama_cpp_endpoint, json=data)

    if response.status_code == 200:
        logger.info(f"{response.json()['content']}")
        logger.info("Инференс через Llama.cpp выполнен успешно.")
        return response.json()['content']
    else:
        logger.error(f"Ошибка инференса через Llama.cpp: {response.status_code}, {response.text}")
        raise Exception(f"Не удалось выполнить инференс через Llama.cpp: {response.status_code}, {response.text}")

# 4. Взаимодействие с внешним сервисом для генерации отчета (официального/неофициального/транскрипции)
@timed_operation("генерации отчета")
def generate_report_with_service(transcription_result: Dict[str, Any], report_type: str, report_service_url: str, output_path: str = "output.docx") -> None:
    """
    Генерирует отчет, отправляя данные в сторонний сервис для формирования DOCX отчета.

    Args:
        transcription_result (Dict[str, Any]): Результаты транскрипции и диаризации.
        report_type (str): Тип отчета: 'official', 'unofficial' или 'transcript'.
        report_service_url (str): URL внешнего сервиса для генерации отчета.
        output_path (str): Путь для сохранения DOCX файла. По умолчанию 'output.docx'.

    Raises:
        Exception: Если запрос к внешнему сервису завершился ошибкой.
    """
    logger.info(f"Отправка данных на создание {report_type} отчета...")

    # Подготовка данных для отправки в сервис
    data = {
        "name_report": transcription_result.get("name_report", "Report"),
        "document_type": "docx",  # Тип документа
        "transcription": transcription_result.get("transcript", ""),
        "diarization": transcription_result.get("diarization", {})
    }

    # Выбор конечной точки для типа отчета
    if report_type == "official":
        report_endpoint = f"{report_service_url}/reports/official"
    elif report_type == "unofficial":
        report_endpoint = f"{report_service_url}/reports/unofficial"
    elif report_type == "transcript":
        report_endpoint = f"{report_service_url}/transcript"
    else:
        raise ValueError("Неверный тип отчета. Доступные типы: 'official', 'unofficial', 'transcript'.")

    # Выполнение POST-запроса на внешний сервис для создания отчета
    response = requests.post(report_endpoint, json=data)

    # Обработка ответа
    if response.status_code == 200:
        logger.info(f"{report_type.capitalize()} отчет успешно создан.")
        # Сохраняем полученный файл как DOCX
        with open(output_path, 'wb') as f:
            f.write(response.content)
        logger.info(f"Документ сохранен как {output_path}")
    else:
        logger.error(f"Ошибка создания {report_type} отчета: {response.status_code}, {response.text}")
        raise Exception(f"Не удалось создать {report_type} отчет: {response.status_code}, {response.text}")



In [ ]:
def get_audio_files_from_folder(audio_path):
    audio_files = []
    for root, dirs, files in os.walk(audio_path):
        for file in files:
            if file.endswith(('.mp4a','.mp3', '.wav', '.ogg')):  # Add formats as needed
                audio_files.append(os.path.join(root, file))
    return audio_files

In [51]:
# Функция для периодической проверки статуса задачи
def check_task_status_periodically(task_id: str, whisperx_status_url: str, poll_interval: int = 5, max_retries: int = 60) -> Dict[str, Any]:
    """
    Периодически проверяет статус задачи по её идентификатору.

    Args:
        task_id (str): Идентификатор задачи транскрипции.
        whisperx_status_url (str): URL для проверки статуса задачи.
        poll_interval (int): Интервал в секундах между запросами статуса. По умолчанию 5 секунд.
        max_retries (int): Максимальное количество попыток проверки статуса. По умолчанию 60 (5 минут).

    Returns:
        Dict[str, Any]: Результат транскрипции, если задача завершена.

    Raises:
        Exception: Если задача завершилась неудачей или превышено количество попыток.
    """
    logger.info(f"Начата периодическая проверка статуса задачи: {task_id['identifier']}")
    status_url = f"{whisperx_status_url}/{task_id['identifier']}"
    logger.info(f"Таска: {status_url}")


    for attempt in range(max_retries):
        response = requests.get(status_url)
        
        if response.status_code == 200:
            result = response.json()
            status = result.get('status', 'unknown')

            if status == 'completed':
                logger.info(f"Задача {task_id['identifier']} завершена успешно.")
                return result['result']
            elif status == 'failed':
                logger.error(f"Задача {task_id['identifier']} завершилась неудачей.")
                raise Exception(f"Задача {task_id['identifier']} завершилась неудачей.")
            else:
                logger.info(f"Задача {task_id['identifier']} ещё не завершена (статус: {status}). Ожидание {poll_interval} секунд...")
                time.sleep(poll_interval)
        else:
            logger.error(f"Ошибка при проверке статуса задачи: {response.status_code}, {response.text}")
            raise Exception(f"Ошибка при проверке статуса: {response.status_code}, {response.text}")
    
    raise Exception(f"Задача {task_id['identifier']} не завершена после {max_retries} попыток.")

In [52]:
@timed_operation("Full pipeline time")
def process_pipeline(audio_path: str, whisperx_url: str, whisperx_status_url: str, llama_cpp_endpoint: str, report_service_url: str, output_path: str = "output.docx") -> None:
    """
    Обрабатывает аудиофайл через весь пайплайн:
    1. Загружает аудио.
    2. Выполняет транскрипцию и диаризацию через WhisperX.
    3. Выполняет инференс через Llama.cpp.
    4. Сохраняет результаты в DOCX файл.

    Args:
        audio_path (str): Путь к аудиофайлу.
        whisperx_url (str): URL для отправки аудио на транскрипцию.
        whisperx_status_url (str): URL для проверки статуса транскрипции.
        llama_cpp_endpoint (str): URL для инференса модели Llama.cpp.
        report_service_url (str): URL для внешнего сервиса отчетов.
        output_path (str): Путь для сохранения DOCX файла. По умолчанию 'output.docx'.
    """
    # Шаг 1: Загрузка аудио
    audio_path = load_audio(audio_path)
    # audio_files = get_audio_files_from_folder(audio_path)


    # Шаг 2: WhisperX Транскрипция и диаризация
    whisperx_params = {'language': 'ru', 'model': 'large-v3'}
    task_id = transcribe_and_diarize(audio_path, whisperx_url, whisperx_params)

    # Шаг 3: Периодическая проверка статуса задачи
    transcription_result = check_task_status_periodically(task_id, whisperx_status_url)

    # Шаг 4: Инференс через Llama.cpp
    llama_result = llama_cpp_inference(transcription_result['segments'], llama_cpp_endpoint)

    # Шаг 5: Генерация отчета через внешний сервис
    generate_report_with_service(transcription_result['segments'], "official", report_service_url, output_path)

In [54]:
# Определение конечных точек (замените на актуальные конечные точки)
### сервис траскрибации
whisperx_endpoint = r"http://127.0.0.1:8004/speech-to-text"
whisperx_endpoint_task = r"http://127.0.0.1:8004/task"

### сервис языковой модели предварительного анализа даннах
llama_cpp_endpoint = r"http://127.0.0.1:34343/completion"

### сервис генерации отчета - транскрипция
report_maker_transcript = r"http://127.0.0.1:8005/transcript"
### сервис генерации отчета - оффициальный
report_maker_off = r"http://127.0.0.1:8005/reports/official"
### сервис генерации отчета - неоффициальный
report_maker_unoff = r"http://127.0.0.1:8005/reports/unofficial"
report_maker= r"http://127.0.0.1:8005"

# тестовый запуск 
audio_path = r"/home/qwerty/Downloads/Telegram Desktop/test_dataset_pravitelstvo_test"
output_path = rf"transcription_{datetime.now().strftime('%Y%m%d_%H%M%S')}.docx"

for 
process_pipeline(audio_path, whisperx_endpoint,whisperx_endpoint_task, llama_cpp_endpoint,report_maker , output_path)

2024-09-08 00:53:07,934 - INFO - Загружен аудиофайл: /home/qwerty/Downloads/Telegram Desktop/train_dataset_pravitelstvo_train/train/Встреча 2. Аудиозапись.mp3
2024-09-08 00:53:07,934 - INFO - Отправка аудиофайла на транскрипцию и диаризацию в WhisperX...
2024-09-08 00:53:08,398 - INFO - Транскрипция и диаризация выполнены успешно.
2024-09-08 00:53:08,400 - INFO - Начата периодическая проверка статуса задачи: c9432c2f-075b-4625-b8b0-17fa8236eb85
2024-09-08 00:53:08,400 - INFO - Таска: http://127.0.0.1:8004/task/c9432c2f-075b-4625-b8b0-17fa8236eb85
2024-09-08 00:53:08,403 - INFO - Задача c9432c2f-075b-4625-b8b0-17fa8236eb85 ещё не завершена (статус: processing). Ожидание 5 секунд...
2024-09-08 00:53:13,406 - INFO - Задача c9432c2f-075b-4625-b8b0-17fa8236eb85 ещё не завершена (статус: processing). Ожидание 5 секунд...
2024-09-08 00:53:18,409 - INFO - Задача c9432c2f-075b-4625-b8b0-17fa8236eb85 ещё не завершена (статус: processing). Ожидание 5 секунд...
2024-09-08 00:53:23,412 - INFO - Зад

AttributeError: 'list' object has no attribute 'get'